In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import json
import os

In [2]:
def find_contours(img):
    # Grayscale image 
    imgray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Threshold the image
    ret, thresh = cv2.threshold(imgray, 127, 255, 0)

    # Find the contours
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # Return an empty array if it did not find any contours
    if len(contours) == 0:
        return np.array([])
    
    # Create a usable list of points from the contours
    contour_points = []

    for contour in contours[0]:
        contour_points.append(contour[0])

    contour_points = np.array(contour_points)

    # Shrink the list of contours 
    shrinked_list = []

    for i in range(0, len(contour_points)):
        if i % 5 == 0: shrinked_list.append(contour_points[i])

    return np.array(shrinked_list)

In [8]:
# Testing contour on a mask
# Create a named colour
img = cv2.imread('masks/6_2.png')
contours = find_contours(img)

red = [0,0,255]

cv2.polylines(img, [contours], True, red, 3)

# Show
cv2.imshow('image', img)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
def get_class(filename):
    num = filename.split('_')[1].split('.')[0]
    
    if num == "1":
        return "one"
    elif num == "2":
        return "two"
    elif num == "3":
        return "three"
    elif num == "4":
        return "four"
    elif num == "5":
        return "five"
    elif num == "6":
        return "six"

In [4]:
class Mask(object):
    def __init__(self, file):
        self.name = file.split('/')[2]
        self.classification = get_class(self.name)
        self.contours = self.get_contours(file)
        
    def get_contours(self, file):
        return find_contours(cv2.imread(file))
        
    
class Image(object):
    def __init__(self, file):
        self.image = cv2.imread(file)
        self.name = file.split('/')[2]
        self.masks = self.get_masks("./masks/" + file.split('/')[2].split('.')[0] + "*.png")
        
    def get_masks(self, path):
        masks = []
        for file in glob.glob(path):
            masks.append(Mask(file))
            
        return masks
        

In [5]:
# Save the points to a json file
images = []

for file in glob.glob("./images/*.png"):
    images.append(Image(file))
    
data = {}

for image in images:
    regions = []
    for mask in image.masks:
        region = {}
        region['name'] = mask.name
        region['type'] = "polyline"
        region['class'] = mask.classification
        region['all_points_x'] = mask.contours.transpose()[0].tolist() if len(mask.contours) > 0 else []
        region['all_points_y'] = mask.contours.transpose()[1].tolist() if len(mask.contours) > 0 else []
        
        regions.append(region)
    
    image_information = {}
    height, width, _ = image.image.shape
    image_information['filename'] = image.name
    image_information['width'] = width
    image_information['height'] = height
    image_information['regions'] = regions
    
    data['image_' + image.name] = image_information
    
with open('annotations.json', 'w') as json_file:
    json.dump(data, json_file)

In [6]:
#For testing

annotations = json.load(open(os.path.join("annotations.json")))
annotations = list(annotations.values())

for annotation in annotations:
    img = cv2.imread("./images/" + annotation['filename'])

    for region in annotation['regions']:    
        points = np.array([region['all_points_x'], region['all_points_y']]).transpose()

        red = [0,0,255]

        cv2.polylines(img, [points], True, red, 2)

    # Show
    cv2.imshow(annotation['filename'], img)

cv2.waitKey(0)
cv2.destroyAllWindows()
    